In [1]:
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from convnetskeras.convnets import preprocess_image_batch, convnet
from skimage.transform import resize
from keras.models import Model
from convnetskeras.imagenet_tool import id_to_synset
import theano
from keras import backend as K

Using Theano backend.


## Using neural net features with random forest

In [2]:
def build_layer_prediction_function(model, input_layer_name=None, output_layer_name=None):
    if input_layer_name is None:
        input_layer = model.layers[0]
    else:
        input_layer = model.get_layer(input_layer_name)
    
    if output_layer_name is None:
        output_layer = model.layers[-1]
    else:
        output_layer = model.get_layer(output_layer_name)

    get_activations = K.function(
        [input_layer.input, K.learning_phase()], 
         output_layer.output)
    def fn(X):
        learning_phase = False
        return get_activations([X, learning_phase])
    return fn

In [3]:
def resize_images(X, w, h):
    X_new = np.empty((X.shape[0], w, h, 3))
    for i in range(X.shape[0]):
        X_new[i] = resize(X[i], (w, h), preserve_range=True)    
    X_new = X_new.transpose((0, 3, 1, 2))
    return X_new

In [4]:
data = np.load("train_64x64.npz")

In [5]:
X, y = data['X'], data['y']

In [6]:
model = convnet('alexnet',weights_path="alexnet_weights.h5", heatmap=False)

In [7]:
for l in model.layers:
    print(l.name)

input_1
conv_1
maxpooling2d_1
convpool_1
zeropadding2d_1
lambda_1
lambda_2
conv_2_1
conv_2_2
conv_2
maxpooling2d_2
lambda_3
zeropadding2d_2
conv_3
zeropadding2d_3
lambda_4
lambda_5
conv_4_1
conv_4_2
conv_4
zeropadding2d_4
lambda_6
lambda_7
conv_5_1
conv_5_2
conv_5
convpool_5
flatten
dense_1
dropout_1
dense_2
dropout_2
dense_3
softmax


In [8]:
predict_dense2 = build_layer_prediction_function(model, output_layer_name='dense_2')

In [10]:
class NeuralTransformer(TransformerMixin):
    
    def __init__(self, fn, batch_size=512):
        self.fn = fn
        self.batch_size = batch_size
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        results = []
        for i in range(0, X.shape[0], self.batch_size):
            results.append(self.fn(resize_images(X[i:i+self.batch_size], 227, 227)))
        return np.concatenate(results, axis=0)

In [11]:
clf = make_pipeline(
    NeuralTransformer(predict_dense2),
    RandomForestClassifier()
)

In [13]:
cross_val_score(clf, X, y, scoring='accuracy', verbose=1, cv=2)

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 51.7min finished


array([ 0.42218511,  0.42389381])

## Check predicted classes in imagenet

In [ ]:
map(id_to_synset, model.predict(resize_images(X[0:10], 227, 227)).argmax(axis=1))
#http://image-net.org/synset?wnid=n01773549